# Q1 - Running Elastic

In [1]:
from elasticsearch import Elasticsearch

In [2]:
!curl localhost:9200

{
  "name" : "6ec83bc34c78",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "mGDfkAN7TGSD4enlKY_yog",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [3]:
# Another way to do it

es_client = Elasticsearch('http://localhost:9200')
es_client.info()

ObjectApiResponse({'name': '6ec83bc34c78', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'mGDfkAN7TGSD4enlKY_yog', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

> ANS: version.build_hash value: "42f05b9372a9a4a470db3b52817899b99a76ee73"

# Q2 - Getting and indexing the data

In [4]:
# Get the FAQ data

import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [5]:
import requests

In [6]:
# Define settings and mappings for a new Elasticsearch index
index_settings = {
    "settings": {
        "number_of_shards": 1,  # Set the number of primary shards for the index to 1
        "number_of_replicas": 0  # Set the number of replica shards for the index to 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},  # Define the 'text' field as a full-text searchable field
            "section": {"type": "text"},  # Define the 'section' field as a full-text searchable field
            "question": {"type": "text"},  # Define the 'question' field as a full-text searchable field
            "course": {"type": "keyword"}  # Define the 'course' field as a keyword field for exact matches
        }
    }
}

# Define the name of the new index
index_name = "course-questions"

# Create a new Elasticsearch index with the specified name and settings
es_client.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/79pgWTPbRq-wyWOi1ceWlg] already exists')

In [7]:
from tqdm.auto import tqdm 

# Iterate over each document in the documents list with a progress bar
for doc in tqdm(documents):
    # Index each document into the specified Elasticsearch index
    es_client.index(index=index_name, document=doc)

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|███████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:22<00:00, 42.07it/s]


> ANS: index() function is used for adding data to elastic.

# Q3 - Searching

In [8]:
query = "How do I execute a command in a running docker container?"

# Define the Elasticsearch search query structure
search_query = {
    "size": 5,  # Limit the search results to the top 5 hits
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,  # User-provided query string
                    "fields": ["question^4", "text"],  # Fields to search with boosting for 'question'
                    "type": "best_fields"  # Use the best field that matches the query for scoring
                }
            },
            # "filter": {
            #     "term": {
            #         "course": "data-engineering-zoomcamp"  # Filter results to only include a specific course
                # }
            # }
        }
    }
}

search_results = es_client.search(index=index_name, body=search_query)


In [9]:
search_results['hits']['hits'][0]['_score']

84.220634

> ANS: 84.05

# Q4 - Filtering

In [10]:
query = "How do I execute a command in a running docker container?"

# Define the Elasticsearch search query structure
search_query = {
    "size": 3,  # Limit the search results to the top 5 hits
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,  # User-provided query string
                    "fields": ["question^4", "text"],  # Fields to search with boosting for 'question'
                    "type": "best_fields"  # Use the best field that matches the query for scoring
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"  # Filter results to only include a specific course
                }
            }
        }
    }
}

search_results = es_client.search(index=index_name, body=search_query)


In [11]:
search_results['hits']['hits'][2]

{'_index': 'course-questions',
 '_id': '1sdhs5EB7yEJzRTdXOBm',
 '_score': 84.220634,
 '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'}}

> ANS: 'How do I copy files from a different folder into docker container’s working directory?'

# Q5. Building a prompt

In [12]:
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [13]:
context_pieces = []

for hit in search_results['hits']['hits']:
    doc = hit['_source']  # Extract the '_source' field from each search hit, which contains the original document data
    context_piece = context_template.format(**doc)  # Unpack the 'doc' dictionary and pass its keys and values as arguments to 'format', which extracts the question and text fields as per the template
    context_pieces.append(context_piece)  # Append the formatted string to the 'context_pieces' list

context = '\n\n'.join(context_pieces)  # Combine all context pieces into a single string with two newlines separating them


`context_template.format(**doc)`: This line replaces the `{question}` and `{text}` placeholders in the context_template with the actual values from the doc dictionary:  

`{question}` is replaced with e.g. 'What is Elasticsearch?'.  

`{text}` is replaced with e.g. 'Elasticsearch is a search engine based on Lucene.'.

In [14]:
prompt = prompt_template.format(question=query, context=context)

In [15]:
prompt

"You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: How do I execute a command in a running docker container?\n\nCONTEXT:\nQ: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I d

In [16]:
len(prompt)

1322

# Q6: Tokens

In [17]:
# !pip install tiktoken

In [18]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [19]:
print(prompt[:100])

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.



In [20]:
len(encoding.encode(prompt))

298

In [21]:
tokens = encoding.encode(prompt)[:10]
tokens

[63842, 261, 4165, 14029, 29186, 13, 30985, 290, 150339, 4122]

In [22]:
encoding.decode(encoding.encode(prompt))

"You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: How do I execute a command in a running docker container?\n\nCONTEXT:\nQ: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I d

In [23]:
encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(prompt)
num_tokens = len(tokens)
print("The number of tokens in the resulting prompt is:", num_tokens)

The number of tokens in the resulting prompt is: 298


# Bonus

In [24]:
# Load environment variables from .env file
# load_dotenv()

import os
from openai import OpenAI

client = OpenAI()

# Get the API key from the environment variable
# api_key = os.getenv('OPENAI_API_KEY')

# Initialize the OpenAI client with the API key
# client = OpenAI(api_key=api_key)

In [25]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [26]:
answer = llm(prompt)

In [28]:
print("Output:", answer)

Output: To execute a command in a running Docker container, you can use the following steps:

1. Identify the container ID of the running container by using:
   ```
   docker ps
   ```

2. Execute the command in the specific container using:
   ```
   docker exec -it <container-id> <command>
   ```

For example, to start a bash session inside the container, you can use:
   ```
   docker exec -it <container-id> bash
   ```


In [29]:
# Print the response
print("Response from OpenAI:")
print(answer)

# Calculate the number of tokens in the response
response_tokens = encoding.encode(answer)
num_response_tokens = len(response_tokens)
print("The number of tokens in the response is:", num_response_tokens)

# Given prices
price_per_1k_input_tokens = 0.005
price_per_1k_output_tokens = 0.015

# Number of requests
number_of_requests = 1000

# Calculate costs
total_input_tokens = num_tokens * number_of_requests
total_output_tokens = num_response_tokens * number_of_requests

cost_for_input_tokens = (total_input_tokens / 1000) * price_per_1k_input_tokens
cost_for_output_tokens = (total_output_tokens / 1000) * price_per_1k_output_tokens
total_cost = cost_for_input_tokens + cost_for_output_tokens

print("Total Input Tokens:", total_input_tokens)
print("Total Output Tokens:", total_output_tokens)
print("Cost for Input Tokens: $", round(cost_for_input_tokens, 2))
print("Cost for Output Tokens: $", round(cost_for_output_tokens, 2))
print("Total Cost for 1000 Requests: $", round(total_cost, 2))

Response from OpenAI:
To execute a command in a running Docker container, you can use the following steps:

1. Identify the container ID of the running container by using:
   ```
   docker ps
   ```

2. Execute the command in the specific container using:
   ```
   docker exec -it <container-id> <command>
   ```

For example, to start a bash session inside the container, you can use:
   ```
   docker exec -it <container-id> bash
   ```
The number of tokens in the response is: 98
Total Input Tokens: 298000
Total Output Tokens: 98000
Cost for Input Tokens: $ 1.49
Cost for Output Tokens: $ 1.47
Total Cost for 1000 Requests: $ 2.96
